# Notebook 1: Data Ingestion, Preparation and Splitting

**Project Overview**
This project predicts dissolved ocean oxygen levels (μmol/kg) from MODIS satellite multispectral features, combining in-situ oceanographic measurements from NOAA World Ocean Database (WOD) with satellite-derived mutlispectral data.

**Notebook Purpose**
This notebook handles initial data ingestion, cleaning, and preparation:
1. Load raw ocean measurement data (NOAA WOD) and MODIS satellite data
2. Clean both datasets (remove duplicates, handle missing values)
3. Merge datasets on latitude, longitude, and date coordinates
4. Perform temporal train/validation/test split (70/15/15)
5. Save clean split datasets for subsequent analysis

**Key Outputs**
- `training set.csv`: Training set (70% of data)
- `validation set.csv`: Validation set (15% of data)
- `testing set.csv`: Test set (15% of data) - locked away until final evaluation

**Data Leakage Prevention**
Data is split temporally (not randomly) to respect temporal autocorrelation in ocean measurements. Test set is saved separately and will not be accessed until Notebook 5.

## Library Imports

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ee
import pickle
import time
import glob
import os
from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Ocean Data

### Data Ingestion

In [ ]:
def parse_ocean_data(file_path):
    """
    Parse WOD oceanographic data file into a clean DataFrame.

    Parameters:
    file_path: String path to the raw NOAA WOD CSV file

    Returns:
    pandas.DataFrame: DataFrame with Cast_id, Latitude, Longitude, Year, Month, Day, Bottom_depth, Depth, and Oxygen columns

    Notes:
    Extracts metadata (cast info, location, date) and depth/oxygen measurements from WOD format
    Removes trailing decimal points with no digits
    Displays progress bar during parsing
    Returns all depth measurements for each cast
    """

    # Metadata fields to extract
    metadata_fields = {
        'CAST': 'Cast_id',
        'Latitude': 'Latitude',
        'Longitude': 'Longitude',
        'Year': 'Year',
        'Month': 'Month',
        'Day': 'Day',
    }

    all_data = []
    current_cast = {}

    print(f"Processing {file_path}...")

    # Get total lines for progress bar
    with open(file_path, 'r') as f:
        total_lines = sum(1 for _ in f)

    with open(file_path, 'r') as f:
        for line in tqdm(f, total=total_lines, desc="Parsing ocean data"):
            parts = [x.strip() for x in line.split(',')]

            # Extract metadata
            for field, col_name in metadata_fields.items():
                if line.startswith(field) and len(parts) >= 3:
                    current_cast[col_name] = parts[2]
                    break

            # Extract depth/oxygen measurements (numbered rows)
            if len(parts) >= 5 and parts[0].isdigit():
                record = current_cast.copy()
                record['Depth'] = parts[1] if parts[1] else None
                record['Oxygen'] = parts[4] if parts[4] else None
                all_data.append(record)

    # Create DataFrame and clean decimals
    df = pd.DataFrame(all_data)

    # Remove trailing decimals with no digits
    for col in df.columns:
        df[col] = df[col].astype(str).str.replace(r'\.$', '', regex=True)

    print(f"Extracted {len(df)} measurements from {df['Cast_id'].nunique()} casts")
    return df

# Usage
file_path = '/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Raw NOAA WOD - OSD 1990-2023.csv'
ocean_df = parse_ocean_data(file_path)

print(f"\nFinal shape: {ocean_df.shape}")
print("Columns:", ocean_df.columns.tolist())

Processing /content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Raw NOAA WOD - OSD 1990-2023.csv...


Parsing ocean data: 100%|██████████| 9430579/9430579 [00:28<00:00, 333569.52it/s]


Extracted 2730185 measurements from 281868 casts

Final shape: (2730185, 8)
Columns: ['Cast_id', 'Latitude', 'Longitude', 'Year', 'Month', 'Day', 'Depth', 'Oxygen']


In [ ]:
ocean_df.head()

,Cast_id,Latitude,Longitude,Year,Month,Day,Depth,Oxygen
0,8087501,36.17,146.0,1990,1,1,0,251
1,8087501,36.17,146.0,1990,1,1,11,251
2,8087501,36.17,146.0,1990,1,1,22,250
3,8087501,36.17,146.0,1990,1,1,25,250
4,8087501,36.17,146.0,1990,1,1,50,247


In [ ]:
# Create Date column from Year, Month, Day
ocean_df['Date'] = pd.to_datetime(ocean_df[['Year', 'Month', 'Day']], errors='coerce')

# Fix variable data types
float_cols = ['Latitude', 'Longitude', 'Depth', 'Oxygen']
int_cols = ['Year', 'Month', 'Day']

ocean_df[float_cols] = ocean_df[float_cols].apply(pd.to_numeric, errors='coerce')
ocean_df[int_cols] = ocean_df[int_cols].apply(pd.to_numeric, errors='coerce').astype('Int64')

# Saved ocean dataset as csv
ocean_df.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Processed NOAA WOD.csv', index=False)
print("Saved processed NOAA WOD data to Google Drive")

ocean_df.head()

Saved processed NOAA WOD data to Google Drive


,Cast_id,Latitude,Longitude,Year,Month,Day,Depth,Oxygen,Date
0,8087501,36.17,146.0,1990,1,1,0.0,251.0,1990-01-01
1,8087501,36.17,146.0,1990,1,1,11.0,251.0,1990-01-01
2,8087501,36.17,146.0,1990,1,1,22.0,250.0,1990-01-01
3,8087501,36.17,146.0,1990,1,1,25.0,250.0,1990-01-01
4,8087501,36.17,146.0,1990,1,1,50.0,247.0,1990-01-01


In [ ]:
# Examine the dataset structure including columns, data types, and non-null values
print("\nDataset Info:")
ocean_df.info()


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2730185 entries, 0 to 2730184
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Cast_id    object        
 1   Latitude   float64       
 2   Longitude  float64       
 3   Year       Int64         
 4   Month      Int64         
 5   Day        Int64         
 6   Depth      float64       
 7   Oxygen     float64       
 8   Date       datetime64[ns]
dtypes: Int64(3), datetime64[ns](1), float64(4), object(1)
memory usage: 195.3+ MB


In [ ]:
# Calculate and display the number of missing values in each column
print("\nMissing Values:")
missing_df = ocean_df.isnull().sum()  # Counts the number of null values in each column to identify data quality issues

print(missing_df)


Missing Values:
Cast_id      0
Latitude     0
Longitude    0
Year         0
Month        0
Day          0
Depth        0
Oxygen       0
Date         0
dtype: int64


In [ ]:
# Count the number of unique values in each column to understand cardinality
print("\nUnique Values:")
ocean_df.nunique()  # Helps identify categorical variables and potential outliers


Unique Values:


,0
Cast_id,281868
Latitude,81985
Longitude,94892
Year,34
Month,12
Day,31
Depth,311291
Oxygen,81219
Date,11185


In [ ]:
# Check for duplicates in the data

# Cast ID duplicates are expected as each cast contains multiple depth measurements
duplicates_cast = ocean_df.duplicated(subset=['Cast_id']).sum()
print(f"Duplicates in ocean data with respect to Cast ID: {duplicates_cast}")

# Location/Date duplicates are expected as casts have measurements at multiple depths (all taken on the same day/ location)
duplicates_location = ocean_df.duplicated(subset=['Latitude', 'Longitude', 'Date']).sum()
print(f"Duplicates in ocean data with respect to Co-ordinates and Date: {duplicates_location}")

# Location/Date/Depth duplicates signify that multiple ships took the same measurements or the same ship took multiple casts or that the data was submitted multiple times
duplicates_full = ocean_df.duplicated(subset=['Latitude', 'Longitude', 'Date', 'Depth']).sum()
print(f"Duplicates in ocean data with respect to Co-ordinates, Date and Depths: {duplicates_full}")

Duplicates in ocean data with respect to Cast ID: 2448317
Duplicates in ocean data with respect to Co-ordinates and Date: 2453934
Duplicates in ocean data with respect to Co-ordinates, Date and Depths: 14812


### Data Processing

In [ ]:
# Address Depth and Date Range Issues

# Limit the ocean data to surface measurements only
ocean_surface = ocean_df.loc[ocean_df.groupby('Cast_id')['Depth'].idxmin()]

# Filter to post-MODIS dates
ocean_surface_post2002 = ocean_surface[ocean_surface['Date'] >= '2002-07-04'].copy()

# Remove duplicate location/date entries
ocean_surface_unique = ocean_surface_post2002.drop_duplicates(
    subset=['Latitude', 'Longitude', 'Date'],
    keep='first'
)

print(f"Original ocean dataset: \t{len(ocean_df):,} rows")
print(f"Surface only measurements: \t{len(ocean_surface):,} rows")
print(f"Post-2002 dataset: \t\t{len(ocean_surface_post2002):,} rows")
print(f"Unique location/dates: \t\t{len(ocean_surface_unique):,} rows")

# Saved ocean dataset as csv
ocean_surface_unique.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Surface NOAA WOD.csv', index=False)
print("Saved surface NOAA WOD data to Google Drive")

ocean_surface_unique.head()

Original ocean dataset: 	2,730,185 rows
Surface only measurements: 	281,868 rows
Post-2002 dataset: 		127,597 rows
Unique location/dates: 		125,050 rows
Saved surface NOAA WOD data to Google Drive


,Cast_id,Latitude,Longitude,Year,Month,Day,Depth,Oxygen,Date
1585713,10181261,59.4970,-5.4980,2002,8,17,9.9,268.0,2002-08-17
1585716,10181262,59.5035,-6.0000,2002,8,17,7.7,264.0,2002-08-17
1585721,10181263,59.5020,-7.0245,2002,8,17,7.8,267.0,2002-08-17
1585759,10181264,59.5020,-8.0090,2002,8,17,73.3,263.0,2002-08-17
1585788,10181265,59.5130,-9.0200,2002,8,17,7.5,268.0,2002-08-17


In [ ]:
# Examine the dataset structure including columns, data types, and non-null values
print("\nDataset Info:")
ocean_surface_unique.info()


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 125050 entries, 1585713 to 2730151
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Cast_id    125050 non-null  object        
 1   Latitude   125050 non-null  float64       
 2   Longitude  125050 non-null  float64       
 3   Year       125050 non-null  Int64         
 4   Month      125050 non-null  Int64         
 5   Day        125050 non-null  Int64         
 6   Depth      125050 non-null  float64       
 7   Oxygen     125050 non-null  float64       
 8   Date       125050 non-null  datetime64[ns]
dtypes: Int64(3), datetime64[ns](1), float64(4), object(1)
memory usage: 9.9+ MB


In [ ]:
# Calculate and display the number of missing values in each column
print("\nMissing Values:")
missing_df = ocean_surface_unique.isnull().sum()  # Counts the number of null values in each column to identify data quality issues

print(missing_df)


Missing Values:
Cast_id      0
Latitude     0
Longitude    0
Year         0
Month        0
Day          0
Depth        0
Oxygen       0
Date         0
dtype: int64


In [ ]:
# Count the number of unique values in each column to understand cardinality
print("\nUnique Values:")
ocean_surface_unique.nunique()  # Helps identify categorical variables and potential outliers


Unique Values:


,0
Cast_id,125050
Latitude,42985
Longitude,47483
Year,22
Month,12
Day,31
Depth,3568
Oxygen,12480
Date,6642


In [ ]:
# Check for duplicates in the data
duplicates = ocean_surface_unique.duplicated(subset=['Latitude', 'Longitude', 'Date']).sum()
print(f"Duplicates in ocean data: {duplicates}")

Duplicates in ocean data: 0


In [ ]:
# Drop 'Depth' and 'Bottom_depth' columns as they are not
ocean_surface_unique = ocean_surface_unique.drop(columns=['Depth'])

# Display the updated columns
print("Remaining Columns")
print(ocean_surface_unique.columns)

Remaining Columns
Index(['Cast_id', 'Latitude', 'Longitude', 'Year', 'Month', 'Day', 'Oxygen',
       'Date'],
      dtype='object')


## Remote Sensing Data

### Data Ingestion

In [ ]:
ee.Authenticate()
ee.Initialize(project='capstone-project-475808')

# Load MODIS Ocean Color collection
modis = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI')

In [ ]:
def extract_modis_row(row):
    """
    Extract MODIS data for a single ocean measurement.

    Parameters:
    row: pandas Series with Latitude, Longitude, date columns

    Returns:
    dict: Dictionary with all MODIS variables or None values if unavailable
    """

    lat = float(row['Latitude'])
    lon = float(row['Longitude'])
    date = pd.to_datetime(row['Date'])

    # Skip pre-MODIS dates
    if date < pd.Timestamp('2002-07-04'):
        return {var: None for var in ['chlor_a', 'poc', 'nflh', 'par', 'sst', 'sst4',
                                       'Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488',
                                       'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645',
                                       'Rrs_667', 'Rrs_678']}

    # Create point geometry
    point = ee.Geometry.Point([lon, lat])

    # Get date range (±3 days for cloud-free composite)
    start = (date - pd.Timedelta(days=3)).strftime('%Y-%m-%d')
    end = (date + pd.Timedelta(days=3)).strftime('%Y-%m-%d')

    # Filter and get median composite
    image = modis.filterDate(start, end).filterBounds(point).median()

    # Extract all values at point
    values = image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=4000
    ).getInfo()

    # Return all variables
    return {
        'chlor_a': values.get('chlor_a'),
        'poc': values.get('poc'),
        'nflh': values.get('nflh'),
        'par': values.get('par'),
        'sst': values.get('sst'),
        'sst4': values.get('sst4'),
        'Rrs_412': values.get('Rrs_412'),
        'Rrs_443': values.get('Rrs_443'),
        'Rrs_469': values.get('Rrs_469'),
        'Rrs_488': values.get('Rrs_488'),
        'Rrs_531': values.get('Rrs_531'),
        'Rrs_547': values.get('Rrs_547'),
        'Rrs_555': values.get('Rrs_555'),
        'Rrs_645': values.get('Rrs_645'),
        'Rrs_667': values.get('Rrs_667'),
        'Rrs_678': values.get('Rrs_678')
    }

# Test Function
# 1. Create a test row directly
test_row = pd.Series({
    'Latitude': 36.0,
    'Longitude': -122.0,
    'Date': pd.to_datetime('2010-06-15')
})

# 2. Get an actual row from your dataframe
# test_row = ocean_df.iloc[100]

# Extract MODIS data
result = extract_modis_row(test_row)
print("Testing extraction for:")
print(f"Location: {test_row['Latitude']}, {test_row['Longitude']}")
print(f"Date: {test_row['Date']}")
print("\nExtracting MODIS data...")

result = extract_modis_row(test_row)

print("\nMODIS Results:")
for key, value in result.items():
    print(f"{key}: {value}")

Testing extraction for:
Location: 36.0, -122.0
Date: 2010-06-15 00:00:00

Extracting MODIS data...

MODIS Results:
chlor_a: 0.9413323998451233
poc: 168.70001220703125
nflh: 0.15487749874591827
par: None
sst: 11.899999618530273
sst4: None
Rrs_412: 0.0019350009970366955
Rrs_443: 0.002178000984713435
Rrs_469: 0.002419000957161188
Rrs_488: 0.002545000985264778
Rrs_531: 0.002219000831246376
Rrs_547: 0.0019520011264830828
Rrs_555: 0.0017400009091943502
Rrs_645: 0.0002220009919255972
Rrs_667: 0.000173000997165218
Rrs_678: 0.0002600009902380407


In [ ]:
def extract_modis_sequential(df, start_idx=0, end_idx=None):
    """
    Extract MODIS data one point at a time.

    Parameters:
    df: pandas.DataFrame with Latitude, Longitude, and Date columns
    start_idx: Starting row index (default: 0)
    end_idx: Ending row index (default: None = process all rows)

    Returns:
    pandas.DataFrame: MODIS data with Latitude, Longitude, Date, and all MODIS variables
    """
    # Filter to post-MODIS dates
    df_filtered = df[df['Date'] >= '2002-07-04'].reset_index(drop=True)

    # Set end index if not provided
    if end_idx is None:
        end_idx = len(df_filtered)

    # Get the subset to process
    subset = df_filtered.iloc[start_idx:end_idx]

    print(f"Processing rows {start_idx} to {end_idx-1} ({len(subset)} points)")

    results = []
    # Process each point
    for idx, row in tqdm(subset.iterrows(), total=len(subset)):
        try:
            # Extract MODIS data
            modis_values = extract_modis_row(row)

            # Add location and date
            modis_values['Latitude'] = row['Latitude']
            modis_values['Longitude'] = row['Longitude']
            modis_values['Date'] = row['Date']

            results.append(modis_values)

        except Exception as e:
            print(f"\nError at row {idx}: {e}")
            continue

    # Convert to DataFrame
    result_df = pd.DataFrame(results)

    print(f"\nExtracted {len(result_df)} points")

    return result_df

In [ ]:
# Process 5000 points at a time, save after each subset
subset_size = 5000

for start in range(0, len(ocean_surface_unique), subset_size):
    end = start + subset_size

    print(f"Subset: {start} to {end}")
    print(f"{'='*60}\n")

    # Extract this subset_size
    subset_df = extract_modis_sequential(
        df=ocean_surface_unique,
        start_idx=start,
        end_idx=end
    )

    # Save this subset immediately
    subset_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/MODIS/subset_{start:06d}_{end:06d}.csv', index=False)

    print(f"Saved subset_{start:06d}_{end:06d}.csv")

    # Brief pause
    time.sleep(10)

print("\nAll Subsets Completed!")

Subset: 0 to 5000

Processing rows 0 to 4999 (5000 points)


  0%|          | 15/5000 [00:14<1:19:46,  1.04it/s]


KeyboardInterrupt: 

In [ ]:
# Find all subset files
subset_files = sorted(glob.glob('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/MODIS/subset_*.csv'))

print(f"Found {len(subset_files)} subset files\n")

# Read and combine
all_subsets = []
for file in subset_files:
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    all_subsets.append(df)
    print(f"{os.path.basename(file)}: {len(df)} rows")

# Combine
modis_df = pd.concat(all_subsets, ignore_index=True)

print(f"\nTotal: {len(modis_df)} rows")

# Save final file
modis_df.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Extracted MODIS Data.csv', index=False)
modis_df.head()

Found 26 subset files

subset_000000_005000.csv: 5000 rows
subset_005000_010000.csv: 5000 rows
subset_010000_015000.csv: 5000 rows
subset_015000_020000.csv: 5000 rows
subset_020000_025000.csv: 5000 rows
subset_025000_030000.csv: 5000 rows
subset_030000_035000.csv: 5000 rows
subset_035000_040000.csv: 5000 rows
subset_040000_045000.csv: 5000 rows
subset_045000_050000.csv: 5000 rows
subset_050000_055000.csv: 5000 rows
subset_055000_060000.csv: 5000 rows
subset_060000_065000.csv: 5000 rows
subset_065000_070000.csv: 5000 rows
subset_070000_075000.csv: 5000 rows
subset_075000_080000.csv: 5000 rows
subset_080000_085000.csv: 5000 rows
subset_085000_090000.csv: 5000 rows
subset_090000_095000.csv: 5000 rows
subset_095000_100000.csv: 5000 rows
subset_100000_105000.csv: 5000 rows
subset_105000_110000.csv: 5000 rows
subset_110000_115000.csv: 5000 rows
subset_115000_120000.csv: 5000 rows
subset_120000_125000.csv: 5000 rows
subset_125000_130000.csv: 50 rows

Total: 125050 rows


,chlor_a,poc,nflh,par,sst,sst4,Rrs_412,Rrs_443,Rrs_469,Rrs_488,Rrs_531,Rrs_547,Rrs_555,Rrs_645,Rrs_667,Rrs_678,Latitude,Longitude,Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.4970,-5.4980,2002-08-17
1,NaN,NaN,NaN,NaN,13.1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.5035,-6.0000,2002-08-17
2,1.291978,177.600006,NaN,NaN,13.2400,NaN,0.005998,0.005116,0.005096,0.00507,0.004878,0.004566,0.004218,0.000876,0.000764,0.000902,59.5020,-7.0245,2002-08-17
3,NaN,NaN,NaN,NaN,12.7525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.5020,-8.0090,2002-08-17
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.5130,-9.0200,2002-08-17


In [ ]:
# Examine the dataset structure including columns, data types, and non-null values
print("\nDataset Info:")
modis_df.info()


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125050 entries, 0 to 125049
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   chlor_a    50329 non-null   float64       
 1   poc        49692 non-null   float64       
 2   nflh       42956 non-null   float64       
 3   par        0 non-null       float64       
 4   sst        90314 non-null   float64       
 5   sst4       0 non-null       float64       
 6   Rrs_412    50328 non-null   float64       
 7   Rrs_443    50328 non-null   float64       
 8   Rrs_469    50328 non-null   float64       
 9   Rrs_488    50328 non-null   float64       
 10  Rrs_531    50328 non-null   float64       
 11  Rrs_547    50328 non-null   float64       
 12  Rrs_555    50328 non-null   float64       
 13  Rrs_645    50328 non-null   float64       
 14  Rrs_667    50328 non-null   float64       
 15  Rrs_678    50328 non-null   float64       
 16  Latit

In [ ]:
# Calculate and display the number of missing values in each column
print("\nMissing Values:")
missing_df = modis_df.isnull().sum()  # Counts the number of null values in each column to identify data quality issues

print(missing_df)


Missing Values:
chlor_a       74721
poc           75358
nflh          82094
par          125050
sst           34736
sst4         125050
Rrs_412       74722
Rrs_443       74722
Rrs_469       74722
Rrs_488       74722
Rrs_531       74722
Rrs_547       74722
Rrs_555       74722
Rrs_645       74722
Rrs_667       74722
Rrs_678       74722
Latitude          0
Longitude         0
Date              0
dtype: int64


In [ ]:
# Count the number of unique values in each column to understand cardinality
print("\nUnique Values:")
modis_df.nunique()  # Helps identify categorical variables and potential outliers


Unique Values:


,0
chlor_a,48902
poc,12640
nflh,11257
par,0
sst,15837
sst4,0
Rrs_412,14790
Rrs_443,12452
Rrs_469,11096
Rrs_488,9715


In [ ]:
# Check for duplicates in the data
duplicates = modis_df.duplicated(subset=['Latitude', 'Longitude', 'Date']).sum()
print(f"Duplicates in MODIS data with respect to Co-ordinates, Date and Depths: {duplicates}")

Duplicates in MODIS data with respect to Co-ordinates, Date and Depths: 0


### Data Processing

In [ ]:
# Quick diagnostic analysis before imputation

# SST correlations
print("SST Correlation with All Variables (ordered):\n")
sst_correlations = modis_df.corr()['sst'].drop('sst').sort_values(ascending=False)
print(sst_correlations)

# SST temporal autocorrelation
print("\nSST Temporal Autocorrelation:")
modis_df_sorted = modis_df.sort_values(['Latitude', 'Longitude', 'Date'])
modis_df_sorted['sst_lag1'] = modis_df_sorted.groupby(['Latitude', 'Longitude'])['sst'].shift(1)
temporal_corr = modis_df_sorted[['sst', 'sst_lag1']].corr().iloc[0, 1]
print(f"   Lag-1 autocorrelation: {temporal_corr:.3f}\n")

# SST spatial autocorrelation
print("SST Spatial Autocorrelation:")
modis_df['lat_bin'] = (modis_df['Latitude'] * 2).round() / 2
modis_df['lon_bin'] = (modis_df['Longitude'] * 2).round() / 2
spatial_std = modis_df.groupby(['lat_bin', 'lon_bin'])['sst'].std().mean()
overall_std = modis_df['sst'].std()
print(f"   Within-region std: {spatial_std:.3f}")
print(f"   Overall std: {overall_std:.3f}")
print(f"   Ratio: {spatial_std/overall_std:.3f}\n")

# POC correlations
print("POC Correlation with All Variables (ordered):\n")
poc_correlations = modis_df.corr()['poc'].drop('poc').sort_values(ascending=False)
print(poc_correlations)

# POC temporal autocorrelation
print("\nPOC Temporal Autocorrelation:")
modis_df_sorted['poc_lag1'] = modis_df_sorted.groupby(['Latitude', 'Longitude'])['poc'].shift(1)
poc_temporal_corr = modis_df_sorted[['poc', 'poc_lag1']].corr().iloc[0, 1]
print(f"   Lag-1 autocorrelation: {poc_temporal_corr:.3f}\n")

# POC spatial autocorrelation
print("POC Spatial Autocorrelation:")
spatial_poc_std = modis_df.groupby(['lat_bin', 'lon_bin'])['poc'].std().mean()
overall_poc_std = modis_df['poc'].std()
print(f"   Within-region std: {spatial_poc_std:.3f}")
print(f"   Overall std: {overall_poc_std:.3f}")
print(f"   Ratio: {spatial_poc_std/overall_poc_std:.3f}\n")

modis_df = modis_df.drop(columns=['lat_bin', 'lon_bin'])

SST Correlation with All Variables (ordered):

Rrs_412      0.482014
Rrs_443      0.460474
Rrs_469      0.421812
Rrs_488      0.337485
Longitude    0.115500
Date         0.011160
Rrs_531     -0.009551
nflh        -0.046146
Rrs_547     -0.065703
Rrs_555     -0.079888
Rrs_667     -0.087188
Rrs_678     -0.088550
Rrs_645     -0.112956
chlor_a     -0.151841
poc         -0.158989
Latitude    -0.386439
par               NaN
sst4              NaN
Name: sst, dtype: float64

SST Temporal Autocorrelation:
   Lag-1 autocorrelation: 0.682

SST Spatial Autocorrelation:
   Within-region std: 2.055
   Overall std: 7.826
   Ratio: 0.263

POC Correlation with All Variables (ordered):

chlor_a      0.649085
Latitude     0.229346
lat_bin      0.229290
Rrs_645      0.069634
Rrs_678      0.055823
Rrs_667      0.053463
nflh         0.040280
Rrs_555      0.036200
Rrs_547      0.014505
Date         0.013291
Longitude    0.006866
lon_bin      0.006839
Rrs_531     -0.049024
sst         -0.158989
Rrs_488     -0.2

#### Observation:
SST shows no strong correlations with other variables (max r=0.48 with Rrs bands),
making regression-based imputation unsuitable. However, SST demonstrates excellent
spatial consistency (within-region ratio=0.290), so spatial median imputation is recommended
with 0.5° grid cells to leverage regional temperature patterns.

POC shows strong correlation with chlor_a (r=0.649) and excellent spatial consistency
(ratio=0.157). POC imputatuon using linear regression with chlor_a to capture
the biological relationship is recommended, spatial median imputation can be used as a backup for any remaining missing values.

Both variables show weak temporal autocorrelation (<0.5), so temporal imputation methods are not used.

In [ ]:
# Address Missing Values

print(f"\nStarting MODIS rows: {len(modis_df):,}")

# Drop columns that have completely or extensively missing data
print("\nStep 1: Dropping columns with completely or extensive missing data")
print("  Dropping: par (100% missing), sst4 (100% missing), nflh (66% missing)")
modis_df = modis_df.drop(columns=['par', 'sst4', 'nflh']).copy()

# Drop rows without chlor_a (critical feature)
print("\nStep 2: Removing rows without chlor_a")
rows_before = len(modis_df)
modis_df = modis_df[modis_df['chlor_a'].notna()].copy()
print(f"  Rows removed: {rows_before - len(modis_df):,}")
print(f"  Remaining rows: {len(modis_df):,}")

# Keep only rows with complete Rrs bands (all or nothing)
print("\nStep 3: Removing rows without complete Rrs bands")
rrs_columns = [col for col in modis_df.columns if col.startswith('Rrs_')]
rows_before = len(modis_df)
modis_df = modis_df.dropna(subset=rrs_columns).copy()
print(f"  Rows removed: {rows_before - len(modis_df):,}")
print(f"  Remaining rows: {len(modis_df):,}")

# Impute SST using spatial median
print("\nStep 4: Imputing SST (spatial median)")
sst_missing = modis_df['sst'].isna().sum()
print(f"  SST missing values: {sst_missing:,}")

modis_df['lat_bin'] = (modis_df['Latitude'] * 2).round() / 2
modis_df['lon_bin'] = (modis_df['Longitude'] * 2).round() / 2

spatial_sst = modis_df.groupby(['lat_bin', 'lon_bin'])['sst'].transform('median')
modis_df['sst'] = modis_df['sst'].fillna(spatial_sst).fillna(modis_df['sst'].median())

print(f"  SST values imputed: {sst_missing:,}")

# Impute POC using chlor_a regression then spatial median
print("\nStep 5: Imputing POC (chlor_a regression + spatial median)")
poc_missing = modis_df['poc'].isna().sum()
print(f"  POC missing values: {poc_missing:,}")

complete = modis_df[['chlor_a', 'poc']].dropna()
model = LinearRegression()
model.fit(complete[['chlor_a']], complete['poc'])

missing_poc = modis_df['poc'].isna()
modis_df.loc[missing_poc, 'poc'] = model.predict(modis_df.loc[missing_poc, ['chlor_a']])

spatial_poc = modis_df.groupby(['lat_bin', 'lon_bin'])['poc'].transform('median')
modis_df['poc'] = modis_df['poc'].fillna(spatial_poc).fillna(modis_df['poc'].median())

print(f"  POC values imputed: {poc_missing:,}")

modis_df = modis_df.drop(columns=['lat_bin', 'lon_bin'])

# Final verification
print("\nStep 6: Final verification")
modis_clean = modis_df.dropna().copy()
print(f"  Final rows: {len(modis_clean):,}")
print(f"  Data retention: {(len(modis_clean) / 125050) * 100:.1f}%")

# Save cleaned data
modis_clean.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Cleaned MODIS Data.csv', index=False)


Starting MODIS rows: 125,050

Step 1: Dropping columns with completely or extensive missing data
  Dropping: par (100% missing), sst4 (100% missing), nflh (66% missing)

Step 2: Removing rows without chlor_a
  Rows removed: 74,721
  Remaining rows: 50,329

Step 3: Removing rows without complete Rrs bands
  Rows removed: 1
  Remaining rows: 50,328

Step 4: Imputing SST (spatial median)
  SST missing values: 540
  SST values imputed: 540

Step 5: Imputing POC (chlor_a regression + spatial median)
  POC missing values: 637
  POC values imputed: 637

Step 6: Final verification
  Final rows: 50,328
  Data retention: 40.2%


In [ ]:
modis_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50328 entries, 2 to 125013
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   chlor_a    50328 non-null  float64       
 1   poc        50328 non-null  float64       
 2   sst        50328 non-null  float64       
 3   Rrs_412    50328 non-null  float64       
 4   Rrs_443    50328 non-null  float64       
 5   Rrs_469    50328 non-null  float64       
 6   Rrs_488    50328 non-null  float64       
 7   Rrs_531    50328 non-null  float64       
 8   Rrs_547    50328 non-null  float64       
 9   Rrs_555    50328 non-null  float64       
 10  Rrs_645    50328 non-null  float64       
 11  Rrs_667    50328 non-null  float64       
 12  Rrs_678    50328 non-null  float64       
 13  Latitude   50328 non-null  float64       
 14  Longitude  50328 non-null  float64       
 15  Date       50328 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(15)
memory usa

In [ ]:
# Calculate and display the number of missing values in each column
print("\nMissing Values:")
missing_df = modis_clean.isnull().sum()  # Counts the number of null values in each column to identify data quality issues

print(missing_df)


Missing Values:
chlor_a      0
poc          0
sst          0
Rrs_412      0
Rrs_443      0
Rrs_469      0
Rrs_488      0
Rrs_531      0
Rrs_547      0
Rrs_555      0
Rrs_645      0
Rrs_667      0
Rrs_678      0
Latitude     0
Longitude    0
Date         0
dtype: int64


## Data Merging

In [ ]:
# Compare date ranges
print("Date Ranges:")
print(f"Ocean data: {ocean_surface_unique['Date'].min()} to {ocean_surface_unique['Date'].max()}")
print(f"MODIS data: {modis_clean['Date'].min()} to {modis_clean['Date'].max()}")

print("-"*50)

# Compare number of rows
print("\nRow Counts:")
print(f"Ocean data: {len(ocean_surface_unique):,} rows")
print(f"MODIS data: {len(modis_clean):,} rows")
print(f"Difference: {len(ocean_surface_unique) - len(modis_clean):,} rows")

print("-"*50)

# Compare geographic coverage
print("\nGeographic Range:")
print("Ocean data:")
print(f"  Lat: {ocean_surface_unique['Latitude'].min():.2f} to {ocean_surface_unique['Latitude'].max():.2f}")
print(f"  Lon: {ocean_surface_unique['Longitude'].min():.2f} to {ocean_surface_unique['Longitude'].max():.2f}")
print("\nMODIS data:")
print(f"  Lat: {modis_clean['Latitude'].min():.2f} to {modis_clean['Latitude'].max():.2f}")
print(f"  Lon: {modis_clean['Longitude'].min():.2f} to {modis_clean['Longitude'].max():.2f}")

print("-"*50)


Date Ranges:
Ocean data: 2002-07-04 00:00:00 to 2023-11-01 00:00:00
MODIS data: 2002-07-04 00:00:00 to 2022-03-03 00:00:00
--------------------------------------------------

Row Counts:
Ocean data: 125,050 rows
MODIS data: 50,328 rows
Difference: 74,722 rows
--------------------------------------------------

Geographic Range:
Ocean data:
  Lat: -78.64 to 89.99
  Lon: -179.99 to 180.00

MODIS data:
  Lat: -78.57 to 82.13
  Lon: -179.99 to 179.92
--------------------------------------------------


In [ ]:
# Merge Ocean and MODIS Data
merged_df = ocean_surface_unique.merge(
    modis_clean,
    on=['Latitude', 'Longitude', 'Date'],
    how='inner'
)

print(f"Ocean measurements:  {len(ocean_surface_unique):,} rows")
print(f"MODIS measurements:  {len(modis_clean):,} rows")
print(f"Merged dataset:      {len(merged_df):,} rows")

merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Combined Dataset.csv', index=False)
merged_df.head()


Ocean measurements:  125,050 rows
MODIS measurements:  50,328 rows
Merged dataset:      50,328 rows


,Cast_id,Latitude,Longitude,Year,Month,Day,Oxygen,Date,chlor_a,poc,...,Rrs_412,Rrs_443,Rrs_469,Rrs_488,Rrs_531,Rrs_547,Rrs_555,Rrs_645,Rrs_667,Rrs_678
0,10181263,59.5020,-7.0245,2002,8,17,267.0,2002-08-17,1.291978,177.600006,...,0.005998,0.005116,0.005096,0.005070,0.004878,0.004566,0.004218,0.000876,0.000764,0.000902
1,10181268,59.4910,-12.4902,2002,8,18,261.0,2002-08-18,0.459861,115.000000,...,0.005788,0.004936,0.004630,0.004550,0.003218,0.002702,0.002364,0.000376,0.000314,0.000394
2,10181271,59.4970,-15.9770,2002,8,19,271.0,2002-08-19,0.212936,63.400002,...,0.005698,0.004854,0.004384,0.003954,0.002184,0.001736,0.001662,0.000282,0.000208,0.000250
3,10181272,59.5002,-16.9835,2002,8,19,269.0,2002-08-19,0.382912,93.000000,...,0.005042,0.004284,0.004158,0.003726,0.002560,0.002150,0.001976,0.000400,0.000302,0.000318
4,10181273,59.5045,-17.9758,2002,8,19,263.0,2002-08-19,0.262437,72.699997,...,0.004774,0.004309,0.004232,0.003783,0.002159,0.001751,0.001583,0.000267,0.000219,0.000243


In [ ]:
# Examine the dataset structure including columns, data types, and non-null values
print("\nDataset Info:")
merged_df.info()


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50328 entries, 0 to 50327
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Cast_id    50328 non-null  object        
 1   Latitude   50328 non-null  float64       
 2   Longitude  50328 non-null  float64       
 3   Year       50328 non-null  Int64         
 4   Month      50328 non-null  Int64         
 5   Day        50328 non-null  Int64         
 6   Oxygen     50328 non-null  float64       
 7   Date       50328 non-null  datetime64[ns]
 8   chlor_a    50328 non-null  float64       
 9   poc        50328 non-null  float64       
 10  sst        50328 non-null  float64       
 11  Rrs_412    50328 non-null  float64       
 12  Rrs_443    50328 non-null  float64       
 13  Rrs_469    50328 non-null  float64       
 14  Rrs_488    50328 non-null  float64       
 15  Rrs_531    50328 non-null  float64       
 16  Rrs_547    50328 non-null

In [ ]:
# Calculate and display the number of missing values in each column
print("\nMissing Values:")
missing_df = merged_df.isnull().sum()  # Counts the number of null values in each column to identify data quality issues

print(missing_df)


Missing Values:
Cast_id      0
Latitude     0
Longitude    0
Year         0
Month        0
Day          0
Oxygen       0
Date         0
chlor_a      0
poc          0
sst          0
Rrs_412      0
Rrs_443      0
Rrs_469      0
Rrs_488      0
Rrs_531      0
Rrs_547      0
Rrs_555      0
Rrs_645      0
Rrs_667      0
Rrs_678      0
dtype: int64


In [ ]:
# Count the number of unique values in each column to understand cardinality
print("\nUnique Values:")
merged_df.nunique()  # Helps identify categorical variables and potential outliers


Unique Values:


,0
Cast_id,50328
Latitude,23111
Longitude,25284
Year,21
Month,12
Day,31
Oxygen,8422
Date,5738
chlor_a,48901
poc,13253


In [ ]:
# Check for duplicates in the data
duplicates = merged_df.duplicated(subset=['Latitude', 'Longitude', 'Date']).sum()
print(f"Duplicates in Combined data with respect to Co-ordinates, and Date: {duplicates}")

Duplicates in Combined data with respect to Co-ordinates, and Date: 0


## Data Splitting

In [ ]:
# Sort by date
merged_sorted = merged_df.sort_values('Date').reset_index(drop=True)

# Calculate split indices for 70/15/15
n = len(merged_sorted)
train_idx = int(0.70 * n)
val_idx = int(0.85 * n)

# Split data temporally
train_df = merged_sorted.iloc[:train_idx].copy()
val_df = merged_sorted.iloc[train_idx:val_idx].copy()
test_df = merged_sorted.iloc[val_idx:].copy()

# Display split information
print(f"\nTotal samples: {n:,}")
print(f"\nTrain set: {len(train_df):,} ({len(train_df)/n*100:.1f}%)")
print(f"  Date range: {train_df['Date'].min().date()} to {train_df['Date'].max().date()}")

print(f"\nValidation set: {len(val_df):,} ({len(val_df)/n*100:.1f}%)")
print(f"  Date range: {val_df['Date'].min().date()} to {val_df['Date'].max().date()}")

print(f"\nTest set: {len(test_df):,} ({len(test_df)/n*100:.1f}%)")
print(f"  Date range: {test_df['Date'].min().date()} to {test_df['Date'].max().date()}")

# Save splits
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Data Splits/Unprocessed/Training Set.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Data Splits/Unprocessed/Validation Set.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/MADS/SIADS 699/Data/Data Splits/Unprocessed/Testing Set.csv', index=False)


Total samples: 50,328

Train set: 35,229 (70.0%)
  Date range: 2002-07-04 to 2012-07-10

Validation set: 7,549 (15.0%)
  Date range: 2012-07-10 to 2015-08-20

Test set: 7,550 (15.0%)
  Date range: 2015-08-20 to 2022-03-03
